<a href="https://colab.research.google.com/github/mloyorev/Theory/blob/main/DiscretizationAR(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We are going to use the following libraries:


*   ***Numpy:*** It is used to perform operation with scalars and matrices.
*   ***Scipy.special:*** In particular the function **'root_hermite'** will help us to obtain the quadrature nodes and the weights of the quadratures.



In [1]:
import numpy as np
from scipy.special import roots_hermite
from collections import namedtuple

# ***Discretization of AR(1)***

Remember that in **stochastic dynamic programming** we have a stationary shock $s_t$ whose sequence $\left\{s_t\right\}_{t=0}$ satisfies $s_{t+1}=\phi(s_t,ϵ_{t+1})$.

Suppose the case of a AR(1) process of the form...

$$s_{t+1}=ρs_t+(1-ρ)\overline{s}+ϵ_{t+1} ⇒ ϵ_{t+1}=s_{t+1}-ρs_t-(1-ρ)\overline{s}$$

where...
*   $ρ$ represents the persistance of the shock.
*   $\overline{s}$ is the mean of the shocks.
*   $ϵ_{t+1}$ is the stochastic error term.














We suppose that $ϵ_{t+1}$ that is distributed as a normal random variable $N(0, σ^2)$, such that...

$$∫_{-∞}^{∞} \frac{1}{σ\sqrt{2π}} e^{-\frac{1}{2} \left(\frac{ϵ_{t+1}}{σ}\right)^2} dϵ_{t+1}=1$$

if we substitute the value of the stochastic error and if we take into account that $s_t$ is fixed, we have that...

\begin{align*}
∫_{-\infty}^{\infty} \frac{1}{\sigma\sqrt{2\pi}} e^{-\frac{1}{2} \left(\frac{s_{t+1}-\rho s_t-(1-\rho)\overline{s}}{\sigma}\right)^2} d(s_{t+1}-\rho s_t-(1-\rho)\overline{s}) &= 1 \\
∫_{-\infty}^{\infty} \frac{1}{\sigma\sqrt{2\pi}} e^{-\frac{1}{2} \left(\frac{s_{t+1}-\rho s_t-(1-\rho)\overline{s}}{\sigma}\right)^2} ds_{t+1} &= 1
\end{align*}





Tauchen y Hussey propose to replace this integral by...

\begin{align*}
∫_{-\infty}^{\infty}f(s_{t+1}|s_t)ds_{t+1}  &= 1 \\
∫_{-\infty}^{\infty} \frac{f(s_{t+1}|s_t)}{f(s_{t+1}|\overline{s})}f(s_{t+1}|\overline{s})ds_{t+1} &= 1
\end{align*}

If we expand the fraction, then we get...

\begin{align*}
\frac{f(s_{t+1}|s_t)}{f(s_{t+1}|\overline{s})} &= \frac{e^{-\frac{1}{2} \left(\frac{s_{t+1}-\rho s_t-(1-\rho)\overline{s}}{\sigma}\right)^2}}{e^{-\frac{1}{2} \left(\frac{s_{t+1}-\overline{s}}{\sigma}\right)^2}}
\end{align*}





By simplifying the expression and using the linear transformation...

$$z_t=\frac{s_t-\overline{s}}{\sigma\sqrt{2}}$$

we get...

$$\frac{1}{\sqrt{π}}\int_∞^∞e^{-((z_{t+1}-ρz_t)^2-z_{t+1}^2)}e^{-z_{t+1}^2}dz_{t+1} = 1$$




For this integral we can use the ***Gauss-Hermite quadrature***, where $z_i$ are the quadrature nodes and $w_i$ are the weights.

The Gauss-Hermite quadrature approach is a numerical technique used to approximate the definite integral of a function by evaluating it at specific points and weights.

You can notice there are two very important concepts:

*   ***Quadrature node:*** A quadrature node is a specific point within the integration domain at which the function to be integrated is evaluated. In the case of the Gauss-Hermite quadrature the nodes are the zeros of the Hermite polynomial.
*   ***Weights:*** The weight is a coefficient associated with each quadrature node that is used to weight the function evaluated at that node. In the Gauss-Hermite quadrature, the weights are determined from the Hermite polynomial and the exponential weight function $w(x)=e^{-x^2}$.

A ***simpler way of representing the weights*** is that they are the probabilities assigned to each quadrature node.



The quadrature leads to the formula...

$$\frac{1}{\sqrt{π}}∑_{j=1}^{n}w_j Φ(z_j;z_i;\overline{x})≃1$$

In other words, the quantity $w_j Φ(z_j;z_i;\overline{x})$ is an aproximation of the transition probability $π_{ij}$.


Therefore, we can formulate a function for making the process of discretization of AR(1).

In [2]:
from numpy.lib import poly1d
# The function 'tauch-huss' performs the discretization of a AR(1) process using the Tauchen and Hussey method.

def tauch_huss(sbar,    # Mean of the AR(1) process
               rho,     # Persistance of the shock
               sigma,   # Volatility of the shock
               n):      # Number of nodes

    # The fuction roots_hernite compute quadrature nodes (zeros of hernite polynomial) and weights.
    xx, wx = roots_hermite(n)

    # This line calculates the discrete states 's' that represent possible values of discretized AR(1).
    s = np.sqrt(2) * np.sqrt(sigma) * xx + sbar

    # This line creates a 'x' matrix that contains quadrature nodes.
    x = np.tile(xx, (n, 1))

    # Transposes matrix 'x' and creates a new matrix 'z'.
    z = x.T

    # This lines creates a 'w' matrix that contains weights and then transposes it.
    w = np.tile(wx, (n, 1)).T

    # This line computes the transition probabilities.
    # The transition probabilities are determined by the normal distribution
    p = np.exp(z * z - (z - rho * x) * (z - rho * x)) * w / np.sqrt(np.pi)

    # Computes te sum of the transition probailities in each row.
    sx = np.sum(p, axis=1)

    # Guarantees that the sum of each row is equal to 1.
    p1 = p / np.tile(sx, (n, 1)).T

    return s, p, sx, p1

In a more concrete way, this function returns...


*   Discratization of the shocks, $s$.
*   Transition probabilities without guaranteeing that they sum $1$, $p$.
*   Sum of the transtion probabilities without guaranteeing that they sum $1$, $sx$, with the purpose of standardizing the transition probabilities.
*   Transition probabilities without guaranteeing that they sum $1$, $p1$.









******

We could try an example and verify that each row sum exactly $1$

In [3]:
ex_s, ex_p, ex_sx, ex_p1 = tauch_huss(10, 0.5, 0.5, 4)

In [4]:
# Discretization of the shocks
print(ex_s)

[ 8.34931988  9.47535238 10.52464762 11.65068012]


In [5]:
# Transition probabilities without guaranteeing that they sum 1.
print(ex_p)

[[0.35407105 0.1018143  0.01801311 0.00152197]
 [0.54632268 0.55825283 0.32191956 0.09665607]
 [0.09665607 0.32191956 0.55825283 0.54632268]
 [0.00152197 0.01801311 0.1018143  0.35407105]]


In [6]:
# Sum of the transition probabilities that do not guarantee that they sum 1 for each row.
print(ex_sx)

[0.47542043 1.52315114 1.52315114 0.47542043]


In [7]:
# Final transition probabilities.
print(ex_p1)

[[0.74475354 0.21415634 0.0378888  0.00320132]
 [0.35867923 0.36651178 0.21135103 0.06345796]
 [0.06345796 0.21135103 0.36651178 0.35867923]
 [0.00320132 0.0378888  0.21415634 0.74475354]]


In [8]:
# Finally we verify that the final transition probabilities add up to 1
print(np.sum(ex_p1, axis = 1))

[1. 1. 1. 1.]
